In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import shap

#load JS visualization code to notebook
shap.initjs()

In [ ]:
train=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")

In [ ]:
train

In [ ]:
test=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
test.head()

In [ ]:
columns=test.columns[1:]
columns

In [ ]:
target=train["target"]
target

In [ ]:
#There are no missing values
train.isnull().sum().sum(),test.isnull().sum().sum()

In [ ]:
categorical_columns=columns[0:10]
categorical_columns


In [ ]:
numerical_columns=columns[10:]
numerical_columns

In [ ]:
train=train.drop(["id","target"],axis=1)
train

In [ ]:
test=test.drop("id",axis=1)
test

We realize that the categorical variable are ordinal categorical variables so we can use LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
categorical_preprocessor = LabelEncoder()
numerical_preprocessor = StandardScaler()

In [ ]:
"""#Now, we create the transformer and associate each of these preprocessors with their respective columns.
from sklearn.compose import ColumnTransformer

preprocessor=ColumnTransformer([
    ("categorical_encoder",categorical_preprocessor,categorical_columns),
    ("standard_scaler",numerical_preprocessor,numerical_columns)
    
])
"""

In [ ]:
from sklearn.preprocessing import LabelEncoder

for cat in categorical_columns:
    encoder=LabelEncoder()
    encoder.fit(train[cat])
    train[cat]=encoder.transform(train[cat])
    test[cat]=encoder.transform(test[cat])
    
train.head()

I'll do the scaling because i could need to use some linear models or neural network later on.

In [ ]:
from sklearn.preprocessing import StandardScaler

num_encoder=StandardScaler()

In [ ]:
num_encoder.fit(train[numerical_columns])
train[numerical_columns]=num_encoder.transform(train[numerical_columns])
test[numerical_columns]=num_encoder.transform(test[numerical_columns])

In [ ]:
train['cont8_cont0'] = train['cont8']*train['cont0']
train['cont9_cont0'] = train['cont9']*train['cont0']
train['cont9_cont5'] = train['cont9']*train['cont5']
train['cont8_cont5'] = train['cont8']*train['cont5']

test['cont8_cont0'] = test['cont8']*test['cont0']
test['cont9_cont0'] = test['cont9']*test['cont0']
test['cont9_cont5'] = test['cont9']*test['cont5']
test['cont8_cont5'] = test['cont8']*test['cont5']

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    train, target, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()
model.fit(data_train,target_train)

In [ ]:
from sklearn.model_selection import cross_validate

cv_results=cross_validate(model,train,target,cv=5)

In [ ]:
cv_results

In [ ]:
scores=cv_results["test_score"]
print(f" the mean of the scores is equal to : {scores.mean().round(3)} +/- {scores.std():.3f}")

In [ ]:
model.score(data_test,target_test)

%%time
shap_preds = model.predict(data_test, pred_contribs=True)

In [ ]:
predictions1 =model.predict(test)

In [ ]:
predictions1

In [ ]:
sub=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")
sub['target'] = predictions1
sub.to_csv('submission.csv', index=False)

In [ ]:
sub

In [ ]:
X=data_train.values

In [ ]:
explainer=shap.explainers.Linear(model,data_train)
shap_values = explainer.shap_values(data_train)

In [ ]:
np.shape(shap_values)

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(data_test)

In [ ]:
shap.summary_plot(shap_values, data_test)

In [ ]:
import xgboost
from xgboost import XGBRegressor
model = xgboost.XGBRegressor(n_estimators=100)
model.fit(data_train,target_train)

In [ ]:
predictions =model.predict(test)

In [ ]:
predictions

%%time
shap_interactions = model.predict(xgb.DMatrix(test[columns]), pred_interactions=True)

In [ ]:
# Create a tree explainer
xgb_explainer = shap.TreeExplainer(
    model, data_train, feature_names=data_train.columns.tolist()
)

In [ ]:
xgb_explainer

In [ ]:
%%time

# Shap values with tree explainer
shap_values = xgb_explainer.shap_values(data_train, target_train)

In [ ]:
shap_values.shape

In [ ]:
shap_values_xgb = shap_values_xgb[:, :-1]

pd.DataFrame(shap_values_xgb, columns=data_train.columns.tolist()).head()

In [ ]:
interactions = xgb_explainer.shap_interaction_values(data_train, target_train)

In [ ]:
final_predictions=0.7*predictions + 0.3*predictions1
final_predictions

In [ ]:
sub=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")
sub['target'] = final_predictions
sub.to_csv('submission.csv', index=False)

#https://towardsdatascience.com/analysing-interactions-with-shap-8c4a2bc11c2a
explainer = shap.TreeExplainer(model)
shap_interaction = explainer.shap_interaction_values(train)